In [2]:
import pandas as pd

# Load data
df = pd.read_csv('/content/sample_data/dataset.csv')

# Combine date and time into one datetime column
df['DateTime'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

# Compute occupancy ratio
df['OccupancyRatio'] = df['Occupancy'] / df['Capacity']

# Encode VehicleType to numeric weights (bike=0.5, car=1, truck=1.5)
veh_weight = {'bike': 0.5, 'car': 1.0, 'truck': 1.5}
df['VehicleWeight'] = df['VehicleType'].map(veh_weight)

# Encode TrafficConditionNearby: low=0, medium=1, high=2
traffic_map = {'low': 0, 'medium': 1, 'high': 2}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)

# Sort by lot and time for time-series analysis
df = df.sort_values(['SystemCodeNumber', 'DateTime']).reset_index(drop=True)


In [3]:
import numpy as np

# Prepare baseline model DataFrame
df_model1 = df.copy()
df_model1['Price_Model1'] = np.nan
alpha = 5.0
base_price = 10.0

# Compute baseline prices per lot sequentially
for lot, group in df_model1.groupby('SystemCodeNumber'):
    prev_price = base_price
    for idx, row in group.iterrows():
        # Linear update: add alpha * occupancy ratio
        price = prev_price + alpha * row['OccupancyRatio']
        df_model1.at[idx, 'Price_Model1'] = price
        prev_price = price


In [4]:
# Compute raw demand score
alpha, beta, gamma, delta, epsilon = 1.0, 0.5, 0.3, 2.0, 1.0
df_demand = df.copy()
df_demand['Demand'] = (alpha * df_demand['OccupancyRatio']
                       + beta * df_demand['QueueLength']
                       - gamma * df_demand['TrafficLevel']
                       + delta * df_demand['IsSpecialDay']
                       + epsilon * df_demand['VehicleWeight'])

# Normalize demand to [0,1] (min-max)
min_d, max_d = df_demand['Demand'].min(), df_demand['Demand'].max()
df_demand['Demand_Norm'] = (df_demand['Demand'] - min_d) / (max_d - min_d)  # see :contentReference[oaicite:12]{index=12}

# Calculate price: BasePrice * (1 + λ * norm_demand), clamped to [0.5x, 2x] base
lambda_param = 1.0
df_demand['Price_Model2'] = base_price * (1 + lambda_param * df_demand['Demand_Norm'])
df_demand['Price_Model2'] = df_demand['Price_Model2'].clip(base_price*0.5, base_price*2.0)


In [5]:
from math import radians, sin, cos, asin, sqrt

# Pre-compute neighbors for each lot within threshold (1 km)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    dlat, dlon = radians(lat2-lat1), radians(lon2-lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1))*cos(radians(lat2))*sin(dlon/2)**2
    return 2 * R * asin(sqrt(a))

# Unique lot coordinates
lots = df[['SystemCodeNumber','Latitude','Longitude']].drop_duplicates().set_index('SystemCodeNumber')
neighbors = {}
for lot, row in lots.iterrows():
    lat1, lon1 = row
    neighs = []
    for other, row2 in lots.iterrows():
        if lot == other:
            continue
        lat2, lon2 = row2
        if haversine(lat1, lon1, lat2, lon2) <= 1.0:
            neighs.append(other)
    neighbors[lot] = neighs

# Initialize competitive prices from Model 2
df_comp = df_demand.copy()
df_comp['Price_Model3'] = df_comp['Price_Model2']

# Adjust prices per timestamp accounting for neighbors:contentReference[oaicite:18]{index=18}
for t in sorted(df_comp['DateTime'].unique()):
    subset = df_comp[df_comp['DateTime'] == t]
    for idx, row in subset.iterrows():
        lot = row['SystemCodeNumber']
        price = row['Price_Model2']
        occ_ratio = row['OccupancyRatio']
        neighs = neighbors.get(lot, [])
        # Find neighbor prices at this timestamp
        neigh_prices = subset[subset['SystemCodeNumber'].isin(neighs)]['Price_Model2'].values
        if occ_ratio >= 0.9 and len(neigh_prices):
            # Lot nearly full and neighbors cheaper?
            if any(neigh_prices < price):
                price *= 0.90  # reduce by 10%
        elif len(neigh_prices):
            # If all neighbors are more expensive, we can raise price
            if all(neigh_prices > price):
                price *= 1.10  # increase by 10%
        # Clamp to [0.5x, 2x] base
        price = max(base_price*0.5, min(price, base_price*2.0))
        df_comp.at[idx, 'Price_Model3'] = price


In [7]:
import numpy as np

# Create a copy to avoid modifying original
df_model1 = df.copy()

# Initialize price column
df_model1['Price_Model1'] = np.nan

alpha = 5.0
base_price = 10.0

# Compute baseline prices per lot sequentially
for lot_id in df_model1['SystemCodeNumber'].unique():
    lot_data = df_model1[df_model1['SystemCodeNumber'] == lot_id]
    prev_price = base_price

    for idx in lot_data.index:
        occ_ratio = df_model1.at[idx, 'OccupancyRatio']
        new_price = prev_price + alpha * occ_ratio
        df_model1.at[idx, 'Price_Model1'] = new_price
        prev_price = new_price


In [8]:
# Create a copy for demand model
df_model2 = df_model1.copy()

# Demand weights (tuneable)
alpha, beta, gamma, delta, epsilon = 1.0, 0.5, 0.3, 2.0, 1.0

# Compute raw demand
df_model2['Demand'] = (
    alpha * df_model2['OccupancyRatio'] +
    beta * df_model2['QueueLength'] -
    gamma * df_model2['TrafficLevel'] +
    delta * df_model2['IsSpecialDay'] +
    epsilon * df_model2['VehicleWeight']
)

# Normalize demand to [0, 1]
min_demand = df_model2['Demand'].min()
max_demand = df_model2['Demand'].max()
df_model2['Demand_Norm'] = (df_model2['Demand'] - min_demand) / (max_demand - min_demand)

# Apply pricing formula
lambda_val = 1.0
df_model2['Price_Model2'] = base_price * (1 + lambda_val * df_model2['Demand_Norm'])

# Clamp prices to [0.5x, 2x]
df_model2['Price_Model2'] = df_model2['Price_Model2'].clip(lower=base_price * 0.5, upper=base_price * 2.0)


In [9]:
from math import radians, cos, sin, asin, sqrt

# Haversine distance function
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    return 2 * R * asin(sqrt(a))

# Identify neighboring lots within 1km
lots_df = df_model2[['SystemCodeNumber', 'Latitude', 'Longitude']].drop_duplicates()
neighbors = {}

for i, row1 in lots_df.iterrows():
    neighbors[row1['SystemCodeNumber']] = []
    for j, row2 in lots_df.iterrows():
        if row1['SystemCodeNumber'] == row2['SystemCodeNumber']:
            continue
        dist = haversine(row1['Latitude'], row1['Longitude'], row2['Latitude'], row2['Longitude'])
        if dist <= 1.0:
            neighbors[row1['SystemCodeNumber']].append(row2['SystemCodeNumber'])

# Create new price column
df_model2['Price_Model3'] = df_model2['Price_Model2'].copy()

# Adjust prices competitively
for time in df_model2['DateTime'].unique():
    snapshot = df_model2[df_model2['DateTime'] == time]
    for idx, row in snapshot.iterrows():
        lot = row['SystemCodeNumber']
        occ_ratio = row['OccupancyRatio']
        price = row['Price_Model2']
        nearby = neighbors.get(lot, [])
        neighbor_prices = snapshot[snapshot['SystemCodeNumber'].isin(nearby)]['Price_Model2'].values

        if len(neighbor_prices) > 0:
            if occ_ratio >= 0.9 and any(neighbor_prices < price):
                price *= 0.9  # decrease by 10%
            elif all(neighbor_prices > price):
                price *= 1.1  # increa*


In [10]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column

output_notebook()

def plot_lot_prices(lot_id):
    lot_df = df_model2[df_model2['SystemCodeNumber'] == lot_id]
    p = figure(x_axis_type='datetime', title=f"Pricing for {lot_id}", width=800, height=300)
    p.line(lot_df['DateTime'], lot_df['Price_Model1'], color='blue', legend_label='Model 1 (Linear)')
    p.line(lot_df['DateTime'], lot_df['Price_Model2'], color='green', legend_label='Model 2 (Demand)')
    p.line(lot_df['DateTime'], lot_df['Price_Model3'], color='red', legend_label='Model 3 (Competitive)')
    p.legend.location = 'top_left'
    p.xaxis.axis_label = 'Time'
    p.yaxis.axis_label = 'Price ($)'
    return p

# Example usage
show(plot_lot_prices(df_model2['SystemCodeNumber'].unique()[0]))  # plot for first lot
